*****************************************************************
#  The Social Web: data representation
- Instructors: Davide Ceolin, Emma Beauxis-Aussalet.
- TAs: Zubaria Inayat, Maxim Sergeev, Zhuofan Mei, Alexander Schmatz, Ling Jin.
- Exercises for Hands-on session 2
*****************************************************************

In this session you are going to mine data in various microformats. You will see the differences in what each of the formats can contain and what purpose they serve. We will start by looking at geographical data.

Prerequisites:
- Python 3.8
- Python packages: requests, BeautifulSoup4, HTMLParser, rdflib


In [3]:
# If you're using a virtualenv, make sure it's activated before running
# this cell!
%pip install requests
%pip install BeautifulSoup4
%pip install HTMLParser
%pip install rdflib
%pip install cloudscraper


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


##  Exercise 1

Even if web pages do not use microformat, interesting data can often be extracted from the HTML. You may use packages such as BeautifulSoup to extract arbitrary pieces of data from any HTML page.
The example below shows how we can find the URL of first image in the infobox table of the wikipedia page on Amsterdam. Tip: compare the code below with HTML source code of the wikipedia page: the image url is in the "src" attribute of the "img" element of in the "table" element with class="infobox".

In [4]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup

# This script requires you to add a url of a page with geotags to the commandline, e.g.
# python geo.py 'http://en.wikipedia.org/wiki/Amsterdam'
URL = 'https://en.wikipedia.org/wiki/Amsterdam'

req = requests.get(URL, headers={'User-Agent' : "Social Web Course Student"})
soup = BeautifulSoup(req.text)
# print(req.text)
image1 = soup.findAll('table', class_='infobox')[0].find('img')
print(image1['src'])  


//upload.wikimedia.org/wikipedia/commons/thumb/5/57/Imagen_de_los_canales_conc%C3%A9ntricos_en_%C3%81msterdam.png/268px-Imagen_de_los_canales_conc%C3%A9ntricos_en_%C3%81msterdam.png


Extracting coordinates from a webpage and reformatting them in the geo microformat (based on Example 8-1 in Mining the Social Web). Note that wikipages may encode long/lat information in different ways. On of the ways used by the Amsterdam wikipedia page is in a span element that is not shown to the user: 
<span class="geo">52.367; 4.900</span>
This span element has a single child: len(geoTag == 1) and no further structure, we have to manually get the long/lat by splitting the string on the ';' semicolon.

In [5]:

geoTag = soup.find(True, 'geo')
print(geoTag)

if geoTag and len(geoTag) > 1:
        lat = geoTag.find(True, 'latitude').string
        lon = geoTag.find(True, 'longitude').string
        print ('Location is at'), lat, lon
elif geoTag and len(geoTag) == 1:
        (lat, lon) = geoTag.string.split(';')
        (lat, lon) = (lat.strip(), lon.strip())
        print (('Location is at'), lat, lon)
else:
        print ('Location not found')


<span class="geo">52.37278; 4.89361</span>
Location is at 52.37278 4.89361


### Task 1

Can you convert the output of Exercise 1 into KML? Here is the KML documentation: https://developers.google.com/kml/documentation/?csw=1 and here you can find a simple example of how it is used: https://renenyffenegger.ch/notes/tools/Google-Earth/kml/index

Visualise the point in Google Maps using the following code example: https://developers.google.com/maps/documentation/javascript/examples/layer-kml-features
You will have to create your own KML file for the custom map layer, and provide a URL to the KML file inside the JavaScript code, which means that you have to upload the file somewhere. You can use a service like http://pastebin.com/ to obtain a URL for your KML file —> paste the code there and request the RAW format URL; use this one in this Task1. If it fails to work you can also use KML viewer websites like https://kmzview.com/.

Is KML a microformat, why (not)?

## Is KML a microformat?
**Yes**, KML is a microformat.
Microformats are are human first formats built upon existing open data standards, such as XML, and KML is an extension of XML, adding new elements and attributes.

In [6]:
kml = f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Placemark>
    <name>A simple placemark on the ground</name>
    <description>This is dam square.</description>
    <Point>
			<coordinates>{lon},{lat},0</coordinates>
    </Point>
  </Placemark>
</kml>'''

# Write the KML file
f = open('./location.kml', 'w')
f.write(kml)
f.close()

## Exercise 2 
In order to find information in the web we can use microformats such as [hRecipe](https://microformats.org/wiki/hrecipe) or Schema.org's [Recipe](https://schema.org/Recipe). But first, we'll show you how to find arbitrary tags in a webpage.


### Task 2 
Parsing data for a <sub><sup>veggie</sup></sub> spaghetti alla carbonara recipe (from Example 2-7 in Mining the Social Web).

In [10]:
import cloudscraper
import json
from bs4 import BeautifulSoup

# A yummy webpage (feel free to change to your likings.)
URL = "https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara"

# Create a CloudScraper object
scraper = cloudscraper.create_scraper()

# Use the CloudScraper object to fetch the HTML content
response = scraper.get(URL)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Now you can work with the 'soup' object as you did before
listchildren = list(soup.children)
#print(listchildren)


We can find any element in the page through *css tag selectors*
You can find them all [here](https://www.w3schools.com/cssref/css_selectors.asp), but shortly these are "." for classes, # for ids and plain text for the element name.


You can also combine them, so that looking for ".class1.class2" would select all elements displaying both classes. For a deeper overview please check the above link (or google "html tag selectors"). 

In [11]:
print(len(listchildren)) # we can see here how many children the html doc has got.
title_unparsed = soup.select_one("title")
#show the title element
print(title_unparsed)

2
<title>Vegetarian Carbonara – A Couple Cooks</title>


Not so pretty.... Use the text method.

In [9]:
print(title_unparsed.text)

Vegetarian Carbonara – A Couple Cooks


The website has a block of JSON-LD data embedded. Try to see if you can find it in the soup object.
We can load the JSON-LD script to work with it easier.
Lets get a list of the ingredients.

In [12]:
# Find the script tag containing the JSON-LD data
json_ld_script = soup.find("script", {"class": "yoast-schema-graph"})

# Extract the content of the script tag
script_content = json_ld_script.string

# Load the JSON data from the script content
data = json.loads(script_content)

# Access the "recipeIngredient" list
recipe_ingredients = data["@graph"][7]["recipeIngredient"]

# Print the list of ingredients
for ingredient in recipe_ingredients:
    print(ingredient)

1 pound spaghetti noodles
½ cup smoked mozzarella cheese
½ cup grated Parmesan cheese, plus more for serving
4 egg yolks
1 cup frozen Earthbound Farm Organic peas
8 cups Earthbound Farm Organic spinach
3 tablespoons butter
Kosher salt
Fresh ground black pepper


Lets also print out the instructions.

In [13]:
recipe_instructions= data["@graph"][7]["recipeInstructions"]
#the instructions list contains dictionaries as elements, take a look at how the list is organized
for step in recipe_instructions:
    print(step["text"])

In a large pot, combine 6 quarts of water with 2 tablespoons kosher salt and bring it to a boil.
Grate the Parmesan and mozzarella cheese. Carefully separate four egg yolks and set aside.
Once boiling, add the pasta and cook until the pasta is just about al dente, about 7 minutes; then add peas and spinach and cook for 1 minute. Reserve 1 cup cooking water, and then drain the pasta and vegetables.
In a skillet, melt the butter, then stir in the cheeses, ¼ cup pasta water, and ¼ teaspoon kosher salt. Stir in the pasta and vegetables until creamy over low heat, adding more pasta water if necessary (note that the mozzarella will stick together in some places).
To serve, top each pasta serving with a whole egg yolk and additional Parmesan cheese, and stir the yolk into the pasta at the table (if you are uncomfortable serving egg yolks at the table, stir the egg yolks into the pasta in the skillet to heat them through). Serve immediately. (Note that the mozzarella cheese can become gummy th

Websites are going to be structured differently. Look at the following JSON-DL snippet.

In [14]:
json_example = {
    "title": "The anarchist cookbook",
    "recipeInstructions": "<ol class=\"recipeSteps\"><li>Cook the linguine according to the packet instructions. </li><li>Meanwhile, carefully crack the eggs into a small bowl and beat them with a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. </li><li>When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan. </li><li>Stir in the egg mixture and spinach with a wooden spoon – they'll cook gently in the residual heat. Add a little pasta water to loosen, if needed. </li><li>Share between bowls and serve with a green salad.</li></ol>",
    "ingredients": ["a lot of effort", "the right mindset"]
}

recipe_instructions = json_example["recipeInstructions"]
example_soup = BeautifulSoup(recipe_instructions, 'html.parser')

In [15]:
#to get a nice and clean list of the instructions, step by step
#we can use the find method to get the first "ol" element with attribute "class.." and then use find_all to get all list elements in there
#then we can strip the list items to obtain the instructions
list_items = example_soup.find('ol', class_='recipeSteps').find_all('li')
instructions = [item.get_text(strip=True) for item in list_items]
print(instructions)

['Cook the linguine according to the packet instructions.', 'Meanwhile, carefully crack the eggs into a small bowl and beat them with a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest.', 'When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan.', "Stir in the egg mixture and spinach with a wooden spoon – they'll cook gently in the residual heat. Add a little pasta water to loosen, if needed.", 'Share between bowls and serve with a green salad.']


## Task 2.1
Now it's your turn. Create a function that can scrape any recipe webpage from the same website (other websites will have different class tags). 

Make sure to:

- return itemized content (e.g. ingredients) in a list. You may want to use a list comprehension here.
- Not all items have been cleaned of their html markdown (see variables ```ingredients``` vs. ```instructions_unparsed```. Make sure to return a list with human readable content (i.e. by using the ```.text``` attribute).


In [21]:
#Here you can see the solution for our example website

URL = "https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara"

def parse_website(url):
    # Create a CloudScraper object
    scraper = cloudscraper.create_scraper()

    # Use the CloudScraper object to fetch the HTML content
    response = scraper.get(URL)

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Get the title
    title_unparsed = soup.select_one("title")
    fn = title_unparsed.text
    
    json_ld_script = soup.find("script", {"class": "yoast-schema-graph"})

    # Extract the content of the script tag
    script_content = json_ld_script.string

    # Load the JSON data from the script content
    data = json.loads(script_content)

    # Access the "recipeIngredient" list
    recipe_ingredients = data["@graph"][7]["recipeIngredient"]
    
    ingredients = [ingredient for ingredient in recipe_ingredients]
    
    #Access the instructions
    recipe_instructions= data["@graph"][7]["recipeInstructions"]
    #the instructions list contains dictionaries as elements, take a look at how the list is organized
    instructions = [step["text"] for step in recipe_instructions]

    return {'name': fn,
            'ingredients': ingredients,
            'instructions': instructions,
            }
    
recipe = parse_website(URL)
print (recipe)
        

{'name': 'Vegetarian Carbonara – A Couple Cooks', 'ingredients': ['1 pound spaghetti noodles', '½ cup smoked mozzarella cheese', '½ cup grated Parmesan cheese, plus more for serving', '4 egg yolks', '1 cup frozen Earthbound Farm Organic peas', '8 cups Earthbound Farm Organic spinach', '3 tablespoons butter', 'Kosher salt', 'Fresh ground black pepper'], 'instructions': ['In a large pot, combine 6 quarts of water with 2 tablespoons kosher salt and bring it to a boil.', 'Grate the Parmesan and mozzarella cheese. Carefully separate four egg yolks and set aside.', 'Once boiling, add the pasta and cook until the pasta is just about al dente, about 7 minutes; then add peas and spinach and cook for 1 minute. Reserve 1 cup cooking water, and then drain the pasta and vegetables.', 'In a skillet, melt the butter, then stir in the cheeses, ¼ cup pasta water, and ¼ teaspoon kosher salt. Stir in the pasta and vegetables until creamy over low heat, adding more pasta water if necessary (note that the 

In [78]:
# -*- coding: utf-8 -*-

import cloudscraper
import json
from bs4 import BeautifulSoup


# Pass in a URL containing hRecipe, such as
# https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/

URL = "https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/"


def get_list_from_ol(html):
    list = []
    for item in html.find_all('li'):
        list.append(item.text)
    return list
# Parse out some of the pertinent information for a recipe.
# See http://microformats.org/wiki/hrecipe.
def parse_website(url):
    scraper = cloudscraper.create_scraper()
    response = scraper.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_unparsed = soup.select_one("title")
    title = title_unparsed.text

    # Look for json ld data
    json_ld_script = soup.find("script", {"type": "application/ld+json"})

    script_content = json_ld_script.string
    data = json.loads(script_content)

    recipe_ingredients = data["recipeIngredient"]
    
    ingredients = [ingredient for ingredient in recipe_ingredients]
    recipe_instructions = BeautifulSoup(data["recipeInstructions"], 'html.parser')

    instructions = get_list_from_ol(recipe_instructions)
    

    return {'name': title,
            'ingredients': ingredients,
            'instructions': instructions,
            }
    

    
recipe = parse_website(URL)
print (recipe)

{'name': 'Vegetarian carbonara recipe | Jamie Oliver pasta recipes', 'ingredients': ['400 g dried linguine ', '4 large free-range eggs ', '2 tablespoons soft ricotta cheese  ', '1  lemon ', '100 g fresh or frozen peas  ', '100 g baby spinach  '], 'instructions': ['Cook the linguine according to the packet instructions. ', 'Meanwhile, carefully crack the eggs into a small bowl and beat them with \na fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. ', 'When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan. ', 'Stir in the egg mixture and spinach with a wooden spoon – they’ll cook gently in the residual heat. Add a little pasta water to loosen, if needed. ', 'Share between bowls and serve with a green salad.']}


But How can we get information not only from one website,  but from all? 

The answer: microformats.

But rather than extracting with information manually from the schema.org or hRecipe microformats, we can use a package, ```scrape-schema-recipe``` 

Feel free to experiment with it. 

### Task 2.2
hRecipe is a microformat specifically created for recipes.
Can you for example easily compare different dessert recipe ingredients? For inspiration you can look back at the exercises you did in Hands-on session 1 where you compared different sets of tweets.

In [36]:
%pip install scrape_schema_recipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.1/567.1 kB 7.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 9.0 MB/s eta 0:00:00
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
  Created wheel for jstyleson: filename=jstyleson-0.0.2-py3-none-any.whl size=2384 sha256=bb9b91a59e934c8becae9d1cb09f89d845ae1df6dd57b091420a9fdc17c5222f
  Stored in directory: /Users/bazottie/Library/Caches/pip/wheels/12/51/c6/a1e751db88203e11c6d9ffe4683ca3d8c14b1479639bec1006
  Created wheel for lxml: filename=lxml-4.9.3-cp310-cp310-macosx_13_0_arm64.whl size=1539151 sha256=1ed740e5032f66ceacebf1f42ca2163c5d7e3ea080268d1187e1a1e19c9ae0b9
  Stored in directory: /Users/bazottie/Library/Caches/pip/wheels/38/0

In [79]:
import  scrape_schema_recipe
URL = "https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/"


recipe_list = scrape_schema_recipe.scrape(URL, python_objects=True)
recipe = recipe_list[0]

instructionsHtml = BeautifulSoup(recipe['recipeInstructions'], 'html.parser')

instructions = get_list_from_ol(instructionsHtml)

print(recipe['name'])
print(recipe['recipeIngredient'])
print(instructions)

Veggie carbonara
['400 g dried linguine ', '4 large free-range eggs ', '2 tablespoons soft ricotta cheese  ', '1  lemon ', '100 g fresh or frozen peas  ', '100 g baby spinach  ']
['Cook the linguine according to the packet instructions. ', 'Meanwhile, carefully crack the eggs into a small bowl and beat them with \na fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. ', 'When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan. ', 'Stir in the egg mixture and spinach with a wooden spoon – they’ll cook gently in the residual heat. Add a little pasta water to loosen, if needed. ', 'Share between bowls and serve with a green salad.']


/Users/bazottie/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mf2py/parser.py:120: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 120 of the file /Users/bazottie/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mf2py/parser.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  self.__doc__ = BeautifulSoup(doc)


## Exercise 3

Schema.org is one of the most widely used annotations formats. Schema.org is a multipurpose  template that has been created by a consortium consisting of Yahoo!, Google and Microsoft. It can describe entities, events, products etc. Check out the vocabulary specs on Schema.org.

### Task 3

Parsing schema.org microdata. To parse this data you need to install the rdflib-microdata package, which you have done in one of the previous steps.



In [80]:
from rdflib import Graph

# Source: https://www.youtube.com/watch?v=sCU214rbRZ0
# Pass in a URL containing Schema.org microformats
URL = "http://dbpedia.org/resource/Micheal_Jackson"

# Initialize a graph
g = Graph()

# Parse in an RDF file graph dbpedia
result = g.parse(location=URL)

# Loop through first 10 triples in the graph
for index, (sub, pred, obj) in enumerate(g):
    print(sub, pred, obj)
    if index == 10:
        break

http://dbpedia.org/resource/Micheal_Jackson http://www.w3.org/ns/prov#wasDerivedFrom http://en.wikipedia.org/wiki/Micheal_Jackson?oldid=1056738079&ns=0
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageWikiLink http://dbpedia.org/resource/Michael_Jackson
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/property/wikiPageUsesTemplate http://dbpedia.org/resource/Template:R_from_misspelling
http://dbpedia.org/resource/Micheal_Jackson http://www.w3.org/2000/01/rdf-schema#label Micheal Jackson
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageRedirects http://dbpedia.org/resource/Michael_Jackson
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageRevisionID 1056738079
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageLength 68
http://dbpedia.org/resource/Micheal_Jackson http://xmlns.com/foaf/0.1/isPrimaryTopicOf http://en.wikipedia.org/wiki/Micheal_Jackson
http:

In [81]:
# Print the size of the Graph
print(f'Graph has {len(g)} facts')

Graph has 9 facts


In [82]:
# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format='ttl'))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ns1: <http://dbpedia.org/ontology/> .
@prefix ns2: <http://dbpedia.org/property/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://dbpedia.org/resource/Micheal_Jackson> rdfs:label "Micheal Jackson"@en ;
    ns1:wikiPageID 14995602 ;
    ns1:wikiPageLength "68"^^xsd:nonNegativeInteger ;
    ns1:wikiPageRedirects <http://dbpedia.org/resource/Michael_Jackson> ;
    ns1:wikiPageRevisionID 1056738079 ;
    ns1:wikiPageWikiLink <http://dbpedia.org/resource/Michael_Jackson> ;
    ns2:wikiPageUsesTemplate <http://dbpedia.org/resource/Template:R_from_misspelling> ;
    prov:wasDerivedFrom <http://en.wikipedia.org/wiki/Micheal_Jackson?oldid=1056738079&ns=0> ;
    foaf:isPrimaryTopicOf <http://en.wikipedia.org/wiki/Micheal_Jackson> .




### Task 3.1 
Compare the schema.org information about a band on last.fm to the Facebook Open Graph information about the same band from Facebook. What are the differences? Which format do you think supports better interoperability?

In [93]:
import pandas as pd
import extruct
import requests
from w3lib.html import get_base_url
from urllib.parse import urlparse
import pprint
pp = pprint.PrettyPrinter(indent=4)

def extract_metadata(url):
    """Extract all metadata present in the page and return a dictionary of metadata lists. 
    
    Args:
        url (string): URL of page from which to extract metadata. 
    
    Returns: 
        metadata (dict): Dictionary of json-ld, microdata, and opengraph lists. 
        Each of the lists present within the dictionary contains multiple dictionaries.
    """
    
    r = requests.get(url)
    base_url = get_base_url(r.text, r.url)
    metadata = extruct.extract(r.text, 
                               base_url=base_url,
                               uniform=True,
                               syntaxes=['json-ld',
                                         'microdata',
                                         'opengraph'])
    return metadata

LASTFM = "https://www.last.fm/music/Black+Country,+New+Road"
FACEBOOK = "https://www.facebook.com/BlackCountryNewRoad/"
lastfm_meta = extract_metadata(LASTFM)
facebook_meta = extract_metadata(FACEBOOK)

pp.pprint([lastfm_meta, facebook_meta])



[   {   'json-ld': [],
        'microdata': [   {   '@context': 'http://schema.org',
                             '@type': 'MusicGroup',
                             'album': [   {   '@type': 'MusicAlbum',
                                              'byArtist': 'Black Country, New '
                                                          'Road',
                                              'image': 'https://lastfm.freetls.fastly.net/i/u/avatar170s/c3e33a2d86475d3967180137e43edb38.jpg',
                                              'name': 'Glastonbury 2023',
                                              'url': 'https://www.last.fm/music/Black+Country,+New+Road/Glastonbury+2023'},
                                          {   '@type': 'MusicAlbum',
                                              'byArtist': 'Black Country, New '
                                                          'Road',
                                              'image': 'https://lastfm.freetls.fastly.net/i

### Task 3.1 Answer
Last.fm uses the schema.org `MusicGroup` type, which has standardized detailed properties for the specific entity type.
Facebook uses only generic open graph propeties which are document specific and not specific to the semantic object we are looking for.
OpenGraph is a more general format, but it is not as detailed as schema.org and is in the end, very specific to Facebook.
Therefore, last.fm/schema.org supports better interoperability.

### Task 3.2
Explore the various microformats at http://microformats.org/ and compare the output of the exercises with the output of http://microformats.org/. Think about possible microformats you want to support in your final assignment and read up on how to parse them.